In [1]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
#Simple ROI
import cv2

im = cv2.imread("test.jpeg")

roi = cv2.selectROI(im)

print(roi)

im_cropped = im[int(roi[1]):int(roi[1]+roi[3]),
                int(roi[0]):int(roi[0]+roi[2])]

cv2.imshow("Cropped Image", im_cropped)
cv2.waitKey(0)

In [ ]:
#Live Video canny filtering
import cv2
cv2.namedWindow('window')
def low(x):
    print('low:',x)
def high(x):
    print('high:',x)
def nothing(x):
    pass

cv2.createTrackbar('low','window',0,255,nothing)
cv2.createTrackbar('high','window',0,255,nothing)
cv2.createTrackbar('kernel','window',1,10,nothing)
# img = cv2.imread('Aditya_profile.jpg')
capture = cv2.VideoCapture(0)
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
while True:
    _, img = capture.read()
    kernel = cv2.getTrackbarPos('kernel','window')
    img = cv2.blur(img,(kernel,kernel))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x = cv2.getTrackbarPos('low','window')
    y = cv2.getTrackbarPos('high','window')
    edge = cv2.Canny(gray, x, y)
    cv2.imshow('window',edge)
    if cv2.waitKey(1) == 27:
            cv2.destroyAllWindows()
            break

In [ ]:
# Apply Canny Filter to ROI
import cv2
from matplotlib import pyplot as plt
cv2.namedWindow('window')
def nothing(x):
    pass
def sketch(image):
    kernel = cv2.getTrackbarPos('kernel','window')
    img = cv2.blur(image,(3,3))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x = cv2.getTrackbarPos('low','window')
    y = cv2.getTrackbarPos('high','window')
    edge = cv2.Canny(gray, x, y)
    return edge

cam_capture = cv2.VideoCapture(0)
cv2.destroyAllWindows()

_, frame = cam_capture.read()
r = cv2.selectROI(frame)


while True:
    _, frame = cam_capture.read()

    #Rectangle marker
    
    rect_img = frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    sketcher_rect = rect_img
    sketcher_rect = sketch(sketcher_rect)
    #Conversion for 3 channels to put back on original image (streaming)
    sketcher_rect_rgb = cv2.cvtColor(sketcher_rect, cv2.COLOR_GRAY2RGB)

    #Replacing the sketched image on Region of Interest
    frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])] = sketcher_rect_rgb
    cv2.imshow("Sketcher ROI", frame)
    
    k = cv2.waitKey(1)
    if k == 113:
        break
    elif k == 116:
        print('t')
        _, frame = cam_capture.read()
        cv2.imshow("Sketcher ROI", frame)
        r = cv2.selectROI("Sketcher ROI",frame)
    
    elif k == -1:
        pass
    else:
        print(k)


cv2.destroyAllWindows()

In [1]:
(p1,p2)

NameError: name 'p1' is not defined

In [1]:
#dynamic ROI by clicking two points in image
import cv2, sys

cap = cv2.VideoCapture(0)
cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)

# Our ROI, defined by two points
p1, p2 = None, None
state = 0

# Called every time a mouse event happen
def on_mouse(event, x, y, flags, userdata):
    global state, p1, p2
    
    # Left click
    if event == cv2.EVENT_LBUTTONUP:
        # Select first point
        if state == 0:
            p1 = (x,y)
            state += 1
        # Select second point
        elif state == 1:
            p2 = (x,y)
            state += 1
    # Right click (erase current ROI)
    if event == cv2.EVENT_RBUTTONUP:
        p1, p2 = None, None
        state = 0

# Register the mouse callback
cv2.setMouseCallback('Frame', on_mouse)

while cap.isOpened():
    val, frame = cap.read()
    
    # If a ROI is selected, draw it
    if state > 1:
        cv2.rectangle(frame, p1, p2, (255, 0, 0), 5)
    # Show image
    cv2.imshow('Frame', frame)
    # Let OpenCV manage window events
    key = cv2.waitKey(50)
    # If ESCAPE key pressed, stop
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()

error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:1000: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [4]:
p1,p2

((132, 104), (1791, 975))

In [1]:
class VESC:
    ''' 
    VESC Motor controler using pyvesc
    This is used for most electric scateboards.
    
    inputs: serial_port---- port used communicate with vesc. for linux should be something like /dev/ttyACM1
    has_sensor=False------- default value from pyvesc
    start_heartbeat=True----default value from pyvesc (I believe this sets up a heartbeat and kills speed if lost)
    baudrate=115200--------- baudrate used for communication with VESC
    timeout=0.05-------------time it will try before giving up on establishing connection
    
    percent=.2--------------max percentage of the dutycycle that the motor will be set to
    outputs: none
    
    uses the pyvesc library to open communication with the VESC and sets the servo to the angle (0-1) and the duty_cycle(speed of the car) to the throttle (mapped so that percentage will be max/min speed)
    
    Note that this depends on pyvesc, but using pip install pyvesc will create a pyvesc file that
    can only set the speed, but not set the servo angle. 
    
    Instead please use:
    pip install git+https://github.com/LiamBindle/PyVESC.git@master
    to install the pyvesc library
    '''
    def __init__(self, serial_port, percent=.2, has_sensor=False, start_heartbeat=True, baudrate=115200, timeout=0.05, steering_scale = 1.0, steering_offset = 0.0 ):
        
        try:
            import pyvesc
        except Exception as err:
            print("\n\n\n\n", err, "\n")
            print("please use the following command to import pyvesc so that you can also set")
            print("the servo position:")
            print("pip install git+https://github.com/LiamBindle/PyVESC.git@master")
            print("\n\n\n")
            time.sleep(1)
            raise
        
        assert percent <= 1 and percent >= -1,'\n\nOnly percentages are allowed for MAX_VESC_SPEED (we recommend a value of about .2) (negative values flip direction of motor)'
        self.steering_scale = steering_scale
        self.steering_offset = steering_offset
        self.percent = percent
        
        try:
            self.v = pyvesc.VESC(serial_port, has_sensor, start_heartbeat, baudrate, timeout)
        except Exception as err:
            print("\n\n\n\n", err)
            print("\n\nto fix permission denied errors, try running the following command:")
            print("sudo chmod a+rw {}".format(serial_port), "\n\n\n\n")
            time.sleep(1)
            raise
        
    def run(self, angle, throttle):
        self.v.set_servo((angle * self.steering_scale) + self.steering_offset)
        self.v.set_duty_cycle(throttle*self.percent)

In [2]:
v=VESC("/dev/tty.usbmodem3041")

In [4]:
v.run(0,0)